# Tool calls

To get better results it helps to give the LLM more information. One way to do this is to let the LLM all available tools to
retrieve more information.

We use the example for Capitals again. The number of inhabitants was roughly right but not exactly there was a good amount of
halluzination.

We will now give the LLM a tool to look up Inhabitants and Size of the capital.

(*Disclaimer:* This is only for demonstration purposes. Of course this could be a simple database query instead of using an
LLM with a tool-call.)



In [1]:
# Here is the data:
import duckdb

result = duckdb.sql("Select * from 'data/capitals_wikipedia.csv'").df() #converted to dataframe as it looks nicer in Jupyter
result

,Name,Inhabitants,Size
0,Abidjan (former capital; still hosts some gove...,NaN,NaN
1,Abu Dhabi,1010092.0,972.0
2,Abuja,1235880.0,1476.0
3,Accra,2388000.0,20.4
4,Adamstown,40.0,4.6
...,...,...,...
241,Windhoek,431000.0,5133.0
242,Yaoundé,2765568.0,180.0
243,Yaren (de facto),747.0,1.5
244,Yerevan,1096100.0,223.0


## Basic Setup of the agent with

The following creates the sets up the logging, the output model and setsup logging.

In [2]:
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import logfire
import os
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

# setup logging
load_dotenv()
logfire.configure(token=os.getenv("LOGFIRE_TOKEN"))
logfire.info('Example 2 configured!')
logfire.instrument_pydantic_ai()

#setup the LLM
BASE_URL="http://127.0.0.1:1234/v1"
LM_STUDIO_MODEL="openai/gpt-oss-20b"

model = OpenAIChatModel(LM_STUDIO_MODEL, provider=OpenAIProvider(BASE_URL))

# define the output model
class Capital(BaseModel):
    name: str = Field(..., description="The name of the capital")
    country: str = Field(..., description="The country of the capital, as it was asked by the user.")
    num_inhabitants: float = Field(..., description="The number of inhabitants of the capital")
    size_of_capital: float = Field(..., description="The size of the capital in square kilometers")


#setup the agent, ignore the deps type for the moment
agent = Agent(model, output_type=Capital)

#First test without tool
result = await agent.run("What is the size of the capital of Germany?")
result.output

12:20:14.352 Example 2 configured!
12:20:14.390 agent run
12:20:14.391   chat openai/gpt-oss-20b


Logfire project URL: ]8;id=633114;https://logfire-eu.pydantic.dev/joerg-mueller/pydantic-demos\https://logfire-eu.pydantic.dev/joerg-mueller/pydantic-demos]8;;\

Capital(name='Berlin', country='Germany', num_inhabitants=3644826.0, size_of_capital=891.0)

## Add a tool



In [3]:
from pydantic_ai import RunContext

#setup the agent again (to prevent double definitions for the tool)
agent = Agent(model, output_type=Capital)

#Be aware that this is a decorator that uses the real instance, so agent, not Agent
@agent.tool
def query_numbers(ctx: RunContext[None], capital_name: str) -> {}:
    """query the number of inhabitants and size for a capital"""
    row = duckdb.sql(f"Select inhabitants, size from 'data/capitals_wikipedia.csv' where name = '{capital_name}'").fetchone()
    return {'inhabitants': row[0], 'size': row[1]}

result = await agent.run("What is the size of the capital of Germany?")
result.output

12:20:16.705 agent run
12:20:16.706   chat openai/gpt-oss-20b
12:20:17.769   running 1 tool
12:20:17.769     running tool: query_numbers
12:20:17.778   chat openai/gpt-oss-20b


Capital(name='Berlin', country='Germany', num_inhabitants=3677472.0, size_of_capital=891.3)